In [2]:
# we check if the training.json file has all the valid entries
# we manually add an opening [] bracket and replace "}}" with "}},\n" in notepad
import json
import os
file_name = r'../../data/interim/Training_Data.json'
f = open(file_name,'r')
data = {}
print(file_name)
temp = json.load(f)
print(len(temp))
data = {}
for t in temp:
    data[list(t.keys())[0]] = t[list(t.keys())[0]]
print(len(temp),len(data.keys()))
f.close()
f = open(file_name,'w')
json.dump(data,f)
f.close()

# # we again open it to check it once
# f = open(file_name,'r')
# temp = json.load(f)
# print(len(temp.keys()))

../../data/interim/Training_Data.json
1059
1059 997


In [4]:
f = open(file_name,'r')
temp = json.load(f)
for key in temp.keys():
    print(key,temp[key])

01912 {'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_end': 1728, 'frame_start': 1679, 'instance_id': 0, 'signer_id': 9, 'source': 'asl5200', 'split': 'val', 'url': 'https://www.youtube.com/watch?v=jPn0FQxkdBc', 'variation_id': 0, 'video_id': '01912'}
69206 {'bbox': [439, 68, 1366, 1080], 'fps': 25, 'frame_end': -1, 'frame_start': 1, 'instance_id': 1, 'signer_id': 115, 'source': 'aslbrick', 'split': 'train', 'url': 'http://aslbricks.org/New/ASL-Videos/all.mp4', 'variation_id': 0, 'video_id': '69206'}
68001 {'bbox': [126, 4, 1102, 720], 'fps': 25, 'frame_end': -1, 'frame_start': 1, 'instance_id': 2, 'signer_id': 109, 'source': 'valencia-asl', 'split': 'val', 'url': 'https://www.youtube.com/watch?v=Gmj2_uVjhqI', 'variation_id': 1, 'video_id': '68001'}
68720 {'bbox': [100, 15, 526, 359], 'fps': 25, 'frame_end': 69, 'frame_start': 1, 'instance_id': 3, 'signer_id': 113, 'source': 'lillybauer', 'split': 'test', 'url': 'https://www.youtube.com/watch?v=My4w56H6qwk', 'variation_id': 1, 'video_id':

02560 {'bbox': [700, 59, 1705, 1080], 'fps': 25, 'frame_end': -1, 'frame_start': 1, 'instance_id': 5, 'signer_id': 4, 'source': 'signschool', 'split': 'train', 'url': 'https://signstock.blob.core.windows.net/signschool/videos/db_uploads/SignSchool%20Angle-2q4Tzzy0Gio.mp4', 'variation_id': 0, 'video_id': '02560'}
02561 {'bbox': [60, 15, 229, 192], 'fps': 25, 'frame_end': -1, 'frame_start': 1, 'instance_id': 6, 'signer_id': 11, 'source': 'signingsavvy', 'split': 'val', 'url': 'https://www.signingsavvy.com/signs/mp4/1/1098.mp4', 'variation_id': 0, 'video_id': '02561'}
02562 {'bbox': [0, 12, 226, 240], 'fps': 25, 'frame_end': -1, 'frame_start': 1, 'instance_id': 7, 'signer_id': 30, 'source': 'aslpro', 'split': 'train', 'url': 'http://www.aslpro.com/main/a/angle.swf', 'variation_id': 0, 'video_id': '02562'}
02563 {'bbox': [229, 36, 493, 400], 'fps': 25, 'frame_end': -1, 'frame_start': 1, 'instance_id': 8, 'signer_id': 41, 'source': 'aslsearch', 'split': 'train', 'url': 'http://www.aslsearch

In [ ]:
# the following code concatenates the 2d points from each file, we find corresponding gloss and its encoding and an array of corresponding sequences 
# we replace z values with 0 currently
import json
import os
import torch
from torch.utils.data import Dataset, DataLoader
import pickle
import numpy as np

class PoseDataset(Dataset):
    """PoseDataset."""
    def __init__(self, training_data_json, words_pickle, embeddings_pickle, videos_root_dir, transform=None):
        # we go through the entire folder containing the videos and for each video id we find the gloss and for that gloss we find the embedding from the pickle file
        self.embeddings = []
        self.poses = []
        f = open(training_data_json,'r')
        temp = json.load(f)
        video_gloss_mapping = {}
        for entry in temp:
            gloss = entry['gloss']
            for videos in entry['instances']:
                video_gloss_mapping[videos['video_id']] = gloss
        word_embeddings = []
        words = []
        with (open(embeddings_pickle, "rb")) as openfile:
            while True:
                try:
                    word_embeddings.append(pickle.load(openfile))
                except EOFError:
                    break
        with (open(words_pickle, "rb")) as openfile:
            while True:
                try:
                    words.append(pickle.load(openfile))
                except EOFError:
                    break
        for video_name in os.listdir(videos_root_dir):
            gloss = video_gloss_mapping[video_name[:video_name.index('.'):]]
            data = json.load(open(os.path.join(videos_root_dir,video_name)))
            max_count = max([int(i) for i in list(data.keys())])
            actual_count = 0
            temp = []
            # we add counter at the end of each pose
            # we store 0 for z positions
            for count in data.keys():
                # 'face_keypoints_2d', 'hand_left_keypoints_2d', hand_right_keypoints_2d','pose_keypoints_2d'
                # if people array is empty we need to skip that frame
                if len(data[count]['people']) > 0:
                    t = data[count]['people'][0]['pose_keypoints_2d'] + data[count]['people'][0]['hand_left_keypoints_2d'] + data[count]['people'][0]['hand_right_keypoints_2d'] + data[count]['people'][0]['face_keypoints_2d'] + [actual_count / max_count]
                    for i in range(2,len(t),3):
                        t[i] = 0
                    temp.append(t)
                    actual_count += 1
            #print(len(temp),len(temp[0]))
            indices = np.argwhere(words[0] == gloss)[0]
            if len(indices) == 1:
                self.embeddings.append([word_embeddings[0][indices[0]][0]])
                self.poses.append(temp)
            else:
                print(gloss + "out of vocabulary")
        print(len(self.embeddings),len(self.embeddings[0]),len(self.poses),len(self.poses[0]))
        
    def __len__(self):
        return len(self.poses)

    def __getitem__(self, idx):
        sample = {'embedding': torch.FloatTensor(self.embeddings[idx]), 'poses': torch.FloatTensor(self.poses[idx])}
        return sample
    
dataset = PoseDataset("../../Data Collection Module/WLASL_v0.3.json", "../../data/interim/words.pkl", "../../data/interim/embeddings.pkl", "../../data/interim/Training Data/2D Data/")

In [90]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)
for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['embedding'].size(), sample_batched['poses'].size())

AttributeError: 'list' object has no attribute 'size'

In [92]:
print(dataset.__getitem__(0))

{'embedding': [-0.02117], 'poses': [[175.21, 54.4187, 0, 174.575, 116.743, 0, 113.513, 117.391, 0, 80.3871, 194.671, 0, 122.584, 154.419, 0, 231.058, 115.448, 0, 272.626, 190.782, 0, 231.063, 162.212, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 162.855, 43.9943, 0, 189.475, 44.6362, 0, 147.942, 53.7675, 0, 205.051, 54.3722, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 228.079, 161.306, 0, 223.976, 151.206, 0, 214.822, 141.421, 0, 200.934, 141.105, 0, 191.149, 141.105, 0, 215.769, 139.527, 0, 198.724, 136.055, 0, 188.624, 134.793, 0, 180.733, 134.477, 0, 214.506, 146.471, 0, 196.515, 143.63, 0, 190.833, 143.63, 0, 190.202, 143.63, 0, 214.191, 153.731, 0, 196.199, 150.89, 0, 195.252, 151.521, 0, 198.724, 153.1, 0, 211.981, 161.622, 0, 197.462, 159.097, 0, 196.199, 157.834, 0, 198.724, 158.15, 0, 122.011, 154.707, 0, 130.031, 145.454, 0, 143.911, 137.126, 0, 161.184, 139.285, 0, 167.661, 140.827, 0, 142.677, 132.499, 0, 160.876, 133.425, 0, 170.746, 134.9